In [3]:
import os
import re
from datetime import datetime, timedelta

import pandas as pd
import todoist
from bs4 import BeautifulSoup
from quarter_lib.akeyless import get_secrets
TODOIST_TOKEN = get_secrets("todoist/token")
TODOIST_API = TodoistAPI(TODOIST_TOKEN)

CHECKED = "Yes"
UNCHECKED = "No"


def get_labels(labels, df_labels):
    is_checked = UNCHECKED
    label_list = []
    for label_id in labels:
        label = df_labels.loc[df_labels.id == label_id].name
        label_list.append(label)
        if "DONE" in str(label):
            is_checked = CHECKED
            label_list.remove(label)
    return [item for sublist in label_list for item in sublist], is_checked


def clen_api_response(api_response):
    temp_list = []
    for entry in api_response:
        temp_list.append(entry.data)
    return pd.DataFrame(temp_list)


def filter_data(days):
    TODOIST_API.sync()
    df_items = clen_api_response(TODOIST_API.items.all())
    df_items = df_items[df_items.checked == 0]
    df_notes = clen_api_response(TODOIST_API.notes.all())
    df_labels = clen_api_response(TODOIST_API.labels.all())

    start_date = (datetime.today() - timedelta(days=int(days))).strftime("%Y-%m-%d")

    after_start_date = df_items["date_added"] >= start_date
    df_filtered_items = df_items.loc[after_start_date]

    cleared_list = []
    for index, row in df_filtered_items.iterrows():
        comments = None

        row_id = row["id"]
        date_added = row["date_added"]
        content = row["content"]
        priority = row["priority"]
        description = row["description"]
        notes = df_notes[df_notes.item_id == row_id]
        labels, checked = get_labels(df_filtered_items.loc[index, "labels"], df_labels)

        if len(notes) > 0:
            comments = notes["content"].values
        cleared_list.append(
            {
                "id": row_id,
                "checked": checked,
                "date_added": date_added,
                "content": content,
                "priority": int(priority),
                "comments": comments,
                "labels": labels,
                "description": description,
            }
        )

    filtered_dates = pd.DataFrame(cleared_list)

    filtered_dates.sort_values(by="date_added", inplace=True)
    filtered_dates.reset_index(drop=True, inplace=True)

    filtered_dates["temp_date"] = pd.to_datetime(filtered_dates["date_added"])
    filtered_dates["temp_date"] = filtered_dates["temp_date"] + pd.Timedelta("02:00:00")
    filtered_dates["temp_date_string"] = filtered_dates["temp_date"].dt.strftime("%d.%m.%Y %H:%M")
    filtered_dates = filtered_dates.drop(["temp_date", "date_added"], axis=1)
    filtered_dates = filtered_dates.rename(columns={"temp_date_string": "date_added"})
    filtered_dates["content"] = filtered_dates["content"].str.replace('"', "")
    filtered_dates["source"] = "Todoist"
    filtered_dates["rework-comments"] = ""
    filtered_dates = filtered_dates[
        [
            "checked",
            "date_added",
            "content",
            "rework-comments",
            "priority",
            "comments",
            "description",
            "labels",
            "source",
            "id",
        ]
    ]

    match = "^(([1-9]|[0-2]\d|[3][0-1])\.([1-9]|[0]\d|[1][0-2])\.[2][0]\d{2})$|^(([1-9]|[0-2]\d|[3][0-1])\.([1-9]|[0]\d|[1][0-2])\.[2][0]\d{2}\s([1-9]|[0-1]\d|[2][0-3])\:[0-5]\d.*)$"

    for index, row in filtered_dates.iterrows():
        filtered_dates.at[index, "comments"] = " | ".join(row["comments"]) if row["comments"] is not None else ""
        filtered_dates.at[index, "labels"] = " | ".join(row["labels"]) if row["labels"] is not None else ""

        content = row[2]
        if re.match(match, content):
            date_array = content.split(" ")
            date = date_array[0] + " " + date_array[1]
            filtered_dates.at[index, "date_added"] = date
            new_content = content.split(date)[1][2:]
            filtered_dates.at[index, "content"] = new_content
            filtered_dates.at[index, "source"] = "Voice Recorder"
    filtered_dates = filtered_dates.sort_values("date_added")
    return filtered_dates


def format_html(data):
    data["checked"].replace("No", "❌", inplace=True)
    data["checked"].replace("Yes", "✔️", inplace=True)
    soup = BeautifulSoup(data.to_html())
    for i in soup.findAll("th"):
        if i.text.isnumeric():
            i.name = "td"
    return str(soup)

def format_md(data):
    data["checked"].replace("No", "❌", inplace=True)
    data["checked"].replace("Yes", "✔️", inplace=True)
    return data.to_markdown(tablefmt="pipe")
    

def get_grabber_data():
    data = filter_data(3)
    data.drop("id", axis=1, inplace=True)
    return data


In [4]:
df = get_grabber_data()

In [15]:
f = open("demofile2.txt", "w", encoding="utf-8")
f.write(format_md(df))
f.close()

In [4]:
from datetime import datetime
from zoneinfo import ZoneInfo

server_timezone = 

current_time = datetime.now(ZoneInfo("Europe/Berlin")) 
print(current_time.isoformat(timespec='seconds'))


2022-08-18T17:54:23+02:00


In [5]:
current_time.strftime("%Y-%m-%d")

'2022-08-18'